In [1]:
import pandas as pd
from datetime import date

import datetime
from datetime import timedelta
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from prophet import Prophet

Загрузка тренировочного датасета. В процессе разработки все подготовительные работы с датасетом сделаны в другом ноутбуке, здесь мы используем подготовленный датасет

In [2]:
df = pd.read_csv('sp_sales_task\\train_data.csv')

In [3]:
df.head(3)

,Unnamed: 0,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_sales_in_rub,month_year,price_units,price_pred,...,pr_group_id_group,year,month,week,day,holiday,liquidity,top_st_revenue,cluster_kmean,cluster_st
0,0,7,1542,2022-10-20,1,5,825,2022-10,165.0,0.006738,...,1,2022,10,3,20,0,1,0,241.0,3.0
1,1,3,780,2023-01-29,0,4,196,2023-01,49.0,0.018316,...,1,2023,1,6,29,1,1,0,3.0,2.0
2,2,2,240,2023-03-02,0,1,78,2023-03,78.0,0.367879,...,1,2023,3,3,2,0,1,0,385.0,3.0


In [4]:
df = df.rename(columns = {'pr_sales_in_rub' : 'y'})
df['ds'] = pd.to_datetime(df['date'])

In [5]:
df = df.drop(['pr_sales_in_units', 'month_year', 'date', 'price_units', 'price_pred', 'price_units_log'], axis=1)

Разделяем данные по времени на тренировочные и тестовые. 

In [6]:
date_lag = 15
predictions_period = df['ds'].max() - timedelta(date_lag)
train = df.loc[df['ds'] < predictions_period]
test = df.loc[df['ds'] >= predictions_period]

In [7]:
# Проверяем, что разделение прошло успешно
print("Даты тренировочного датасета: с", train['ds'].min(), "по:", train['ds'].max())
print("Даты тренировочного датасета: с", test['ds'].min(), "по:", test['ds'].max())

Даты тренировочного датасета: с 2022-08-01 00:00:00 по: 2023-07-02 00:00:00
Даты тренировочного датасета: с 2023-07-03 00:00:00 по: 2023-07-18 00:00:00


In [8]:
# Создим признаки и целевые признаки.
X_train = train.drop(['y', 'ds'], axis=1)
y_train = train['y']

In [9]:
# Перебор очень долгий. В связи с тем, что мы пока тестируем модели, используем дефолтные параметры.

# %%time

# # Попробуем изменить гиперпараметры. Будем использовать кросс-валидацию с использованием 
# # библиотеки TimeSeriesSplit, что исключит подглядывание 
# model_cbr_tune = CatBoostRegressor(loss_function='RMSE')
# tscv = TimeSeriesSplit(n_splits=5)
# parameters_cbr_tune = {'iterations': [1000],
#         'learning_rate': [0.03],#, 0.1],
#         'depth': [6],#, 8],
#         'l2_leaf_reg': [1]#, 3]
#               }
# model_cbr_tune = GridSearchCV(model_cbr_tune, 
#                         parameters_cbr_tune, 
#                         cv = tscv, 
#                         n_jobs=-1,
#                               scoring = 'neg_mean_squared_error',
#                              verbose=True).fit(X_train, y_train, 
#                                                verbose=False, plot=True)


Для корректного создания резолютирующего файла предсказания делаем по всем комбинациям магазин-товар. 

In [10]:
model_cbr_tune = CatBoostRegressor(loss_function='RMSE').fit(X_train, y_train, 
                                               verbose=False, plot=True)


model_cbr_tune.get_feature_importance(prettified=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

,Feature Id,Importances
0,pr_subcat_id,19.713059
1,pr_sku_id,15.431507
2,cluster_kmean,14.195850
3,pr_cat_id,9.669312
4,month,7.857134
5,pr_group_id,6.196670
6,day,4.344085
7,pr_group_id_group,3.818077
8,pr_sales_type_id,3.751957
9,week,3.438765


In [11]:
# Определяем параметры перебора по связкам магазин-товар.
a = set(test['st_id'])
b = set(test['pr_sku_id'])

In [12]:
%%time
sales_submission_test = pd.DataFrame(columns=['st_id', 'pr_sku_id', 'date']) #Для вычисления метрики
for i in a:
    for j in b:
        test_pred = test[(test['st_id'] == i) & (test['pr_sku_id'] == j)] # Выделяем часть датасета в разрезе нужной связки
       
        df_1 = pd.DataFrame(columns=['st_id', 'pr_sku_id', 'date', 'y']) # Резолютирующий датасет по предсказаниям

        df_1['st_id'], df_1['pr_sku_id'], df_1['date'], df_1['y'] = \
            test_pred['st_id'], test_pred['pr_sku_id'], test_pred['ds'], test_pred['y']
       
        X_test = test_pred.drop(['y', 'ds'], axis=1) # Выделяем данные для предсказаний

        df_1['target'] = model_cbr_tune.predict(X_test) # Предсказание

        sales_submission_test = pd.concat([sales_submission_test, df_1], ignore_index=False)

<timed exec>:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


CPU times: total: 7.95 s
Wall time: 33.9 s


In [13]:
# sales_submission_test.groupby('date').count()

In [14]:
# Вычисляем метрику
wape = 100 * (sales_submission_test['y'] - sales_submission_test['target']).abs().sum() / sales_submission_test['y'].sum()
wape

61.24550863917664

In [15]:
# Посмотрим, что получилось
sales_submission_test.reset_index(drop= True , inplace= True )
sales_submission_test

,st_id,pr_sku_id,date,y,target
0,1,1,2023-07-18,80.0,277.190028
1,1,1,2023-07-13,81.0,174.721142
2,1,1,2023-07-14,153.0,206.995318
3,1,1,2023-07-15,155.0,255.759112
4,1,1,2023-07-16,156.0,206.467179
...,...,...,...,...,...
38631,9,1969,2023-07-07,69.0,214.245603
38632,9,1969,2023-07-10,300.0,76.363316
38633,9,1969,2023-07-17,82.0,33.709752
38634,9,1969,2023-07-06,81.0,60.814343


In [16]:
# Создаем нужный файл в соответствии с заданием и записываем его

sales_submission = sales_submission_test.drop(['y'], axis=1)
sales_submission.to_csv('sales_submission.csv')